In [2]:
import gymnasium as gym
import numpy as np
import random
from collections import deque
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import warnings

# Wyłączenie ostrzeżeń (dla czytelności konsoli)
warnings.filterwarnings('ignore')

# Ustawienie losowości, by uzyskać powtarzalne wyniki
np.random.seed(42)
tf.random.set_seed(42)

# Tworzymy środowisko CartPole
env = gym.make('CartPole-v1')

# Rozmiar wejścia (stan gry) oraz liczba możliwych akcji (lewo/prawo)
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

# Parametry uczenia
episodes = 50             # Liczba odcinków do trenowania
batch_size = 32           # Wielkość paczki doświadczeń do uczenia
gamma = 0.95              # Współczynnik dyskontowania przyszłych nagród
epsilon = 1.0             # Początkowy współczynnik eksploracji (losowe decyzje)
epsilon_min = 0.01        # Minimalny współczynnik eksploracji
epsilon_decay = 0.995     # Tempo zmniejszania eksploracji

# Tworzymy pamięć do przechowywania doświadczeń
memory = deque(maxlen=2000)

# Funkcja budująca model sieci neuronowej
def build_model(state_size, action_size):
    model = Sequential()
    model.add(Input(shape=(state_size,)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model

model = build_model(state_size, action_size)

# Zapamiętywanie doświadczeń
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

# Wybór akcji na podstawie strategii epsilon-greedy
def act(state):
    if np.random.rand() <= epsilon:
        return random.randrange(action_size)
    q_values = model.predict(state, verbose=0)
    return np.argmax(q_values[0])

# Trenowanie modelu na podstawie doświadczeń (Replay memory)
def replay(batch_size):
    global epsilon
    minibatch = random.sample(memory, batch_size)

    states = np.vstack([exp[0] for exp in minibatch])
    actions = [exp[1] for exp in minibatch]
    rewards = [exp[2] for exp in minibatch]
    next_states = np.vstack([exp[3] for exp in minibatch])
    dones = [exp[4] for exp in minibatch]

    q_next = model.predict(next_states, verbose=0)
    q_target = model.predict(states, verbose=0)

    for i in range(batch_size):
        target = rewards[i]
        if not dones[i]:
            target += gamma * np.amax(q_next[i])
        q_target[i][actions[i]] = target

    model.fit(states, q_target, epochs=1, verbose=0)

    # Zmniejszenie epsilon (coraz mniej losowych ruchów)
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

# Główna pętla treningowa
for e in range(episodes):
    state, _ = env.reset(seed=42)
    state = np.reshape(state, [1, state_size])

    for time in range(200):  # maksymalna liczba kroków w epizodzie
        action = act(state)

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # Dostosowanie kształtu stanu
        next_state = np.reshape(next_state, [1, state_size])

        # Zapamiętanie doświadczenia
        remember(state, action, reward, next_state, done)

        state = next_state

        if done:
            print(f"Odcinek: {e+1}/{episodes}, Wynik: {time}, Epsilon: {epsilon:.2f}")
            break

    # Trenowanie modelu po każdym odcinku, jeśli pamięć jest odpowiednio duża
    if len(memory) >= batch_size:
        replay(batch_size)

env.close()


Odcinek: 1/50, Wynik: 55, Epsilon: 1.00
Odcinek: 2/50, Wynik: 29, Epsilon: 0.99
Odcinek: 3/50, Wynik: 30, Epsilon: 0.99
Odcinek: 4/50, Wynik: 40, Epsilon: 0.99
Odcinek: 5/50, Wynik: 11, Epsilon: 0.98
Odcinek: 6/50, Wynik: 22, Epsilon: 0.98
Odcinek: 7/50, Wynik: 46, Epsilon: 0.97
Odcinek: 8/50, Wynik: 12, Epsilon: 0.97
Odcinek: 9/50, Wynik: 34, Epsilon: 0.96
Odcinek: 10/50, Wynik: 37, Epsilon: 0.96
Odcinek: 11/50, Wynik: 9, Epsilon: 0.95
Odcinek: 12/50, Wynik: 37, Epsilon: 0.95
Odcinek: 13/50, Wynik: 17, Epsilon: 0.94
Odcinek: 14/50, Wynik: 27, Epsilon: 0.94
Odcinek: 15/50, Wynik: 12, Epsilon: 0.93
Odcinek: 16/50, Wynik: 8, Epsilon: 0.93
Odcinek: 17/50, Wynik: 11, Epsilon: 0.92
Odcinek: 18/50, Wynik: 11, Epsilon: 0.92
Odcinek: 19/50, Wynik: 10, Epsilon: 0.91
Odcinek: 20/50, Wynik: 18, Epsilon: 0.91
Odcinek: 21/50, Wynik: 28, Epsilon: 0.90
Odcinek: 22/50, Wynik: 41, Epsilon: 0.90
Odcinek: 23/50, Wynik: 9, Epsilon: 0.90
Odcinek: 24/50, Wynik: 13, Epsilon: 0.89
Odcinek: 25/50, Wynik: 19, E